In [1]:
import os
import json
from pathlib import Path

In [2]:
# object_detection_dir = 'C:/Users/User/0612_Merge_Json/Test_Image_Complete'
object_detection_dir = 'D:/styler/img/0719NET_top/obde json'
# classification_dir = 'C:/Users/User/0612_Merge_Json/Test_Image_Cut'
classification_dir = 'D:/styler/img/0719NET_top/classify json'
# webcrawler_dir = 'C:/Users/User/0612_Merge_Json/webcrawler'
webcrawler_dir = 'D:/styler/img/0719NET_top/web json'
# destination_dir = 'C:/Users/User/0612_Merge_Json/Merge_Json'
destination_dir = 'D:/styler/img/0719NET_top/merge json'
ODList = os.listdir(object_detection_dir)
CList = os.listdir(classification_dir)
WList = os.listdir(webcrawler_dir)

In [4]:
strWList = []#存爬蟲的list
strList = []#存物件偵測的list
strCList = []#存分類器的list
all_dict = {}#dictionary
s = {}#dictionary

for file in ODList:            
    if file.endswith(".json"):
        with open(os.path.join(object_detection_dir,file), 'r') as reader:
            ODjf = json.loads(reader.read())
        jf = str(ODjf).strip('}').strip('{')
        newjf = str(jf).split(":")
        # org_name為檔名，ex:NET_1.json
        org_name = file.split(".")[0]
        
        #把原爬蟲的json檔{"brand":"NET","product_name":"女裝粉紅外套","gender":"w","price":"600","website_url":"https://www.uniqlo.com/tw/store/goods/436738"}
        #拆為"brand","NET","product_name","女裝粉紅外套","gender","w","price","600","website_url","https://www.uniqlo.com/tw/store/goods/436738"
        #這邊有一個問題，用split(":")切割的話，網址也會被切掉，所以後面也會在合併
        #存入strWList中，並且把前面有空格的地方都取刪掉
        for file in WList:    
            name = file.split(".")#ex:NET_1.json
            if file.endswith(".json") and name[0] == org_name:
                with open(os.path.join(webcrawler_dir,file), 'r',encoding = 'utf-8') as reader:#編碼要用utf-8才能存中文字
                    Wjf = json.loads(reader.read())
                jf = str(Wjf).strip('}').strip('{')
                new1jf = str(jf).split(":")
                
                #new1jf[9]=new1jf[9]+new1jf[10]
                for i in range(0,len(new1jf)):
                    if ',' in str(new1jf[i]) :
                        cut = str(new1jf[i]).split(",")                  
                        strWList.append(cut[0].lstrip())
                        strWList.append(cut[1].lstrip())
                    else:
                        strWList.append(new1jf[i].lstrip())
        
        #把原物件偵測的json檔{"filename":"NET_1.jpg","item1":{"Class":"long sleeve top", "Probability": 0.7503372430801392, "Bounding box":[0.06489083, 0.28543484, 0.9990295, 0.96235776]},"item2":{"Class":"trousers", "Probability": 0.6078929901123047, "Bounding box":[0.1509296, 0.7262472, 0.738734, 1.0000496]}}
        #拆為"filename","NET_1.jpg","item1","Class","long sleeve top","Probability",0.7503372430801392,...,"Bounding box",0.1509296,0.7262472,0.738734,1.0000496
        #存入strList中，並且把前面有空格的地方都取刪掉
        for i in range(0,len(newjf)):
            if ',' in str(newjf[i]) and '[' not in str(newjf[i]):
                cut = str(newjf[i]).split(",")
                strList.append(cut[0].lstrip())
                strList.append(cut[1].lstrip())
            elif '}' in str(newjf[i]):
                cut = str(newjf[i]).split("}")
                strList.append(cut[0].lstrip())
                cut_1 = cut[1].split(",")
                strList.append(cut_1[1].lstrip())
            elif '[' in str(newjf[i]) and '}' not in str(newjf[i]):    
                strList.append(newjf[i].lstrip())
            elif '{' in str(newjf[i]):
                cut = str(newjf[i]).split("{")
                strList.append(cut[1].lstrip())
            else:
                strList.append(newjf[i].lstrip())  
        
        #把原分類器的json檔{"Classification":"floral","Highest_probability":0.5591356158256531,"Color":"red"}
        #拆為"Classification","floral","Highest_probability",0.5591356158256531,"Color","red"
        #存入strCList中，並且把前面有空格的地方都取刪掉
        for file in CList:    
            # name[0]+"_"+name[1]判斷是哪一張圖切下來的 、 item[0]判斷是哪一個 item
            name = file.split("_")#ex:NET_1_long sleeve top.json
            name = name[0]+"_"+name[1] #ex:NET_1
            cat = file.split("_")
            #print(name)
            if file.endswith(".json") and name == org_name:
                with open(os.path.join(classification_dir,file), 'r') as reader:
                    Cjf = json.loads(reader.read())
                jf = str(Cjf).strip('}').strip('{')
                newjf = str(jf).split(":")  
                item = cat[2].split(".")#判斷是哪個類別，ex:long sleeve top
                strCList.append(item[0].lstrip())
                for i in range(0,len(newjf)):
                    if ',' in str(newjf[i]) :
                        cut = str(newjf[i]).split(",")                  
                        strCList.append(cut[0].lstrip())
                        strCList.append(cut[1].lstrip())
                    else:
                        strCList.append(newjf[i].lstrip())
        print(strWList)
        #print(strList)
        #print(strCList)
        
        #把"filenam"當成all_dict的key；對應的value是真正的檔名strList[1]，ex:"NET_1.jpg"
        all_dict[strList[0]] = strList[1]  
           
        for j in range(0,1):
            strWList[9] = strWList[9]+':'+strWList[10]#把切錯的網址合併
            all_dict[strWList[0]] = strWList[1]#把"brand"當成all_dict的key；對應的value為"NET"
            all_dict[strWList[2]] = strWList[3]#把"product_name"當成all_dict的key；對應的value為"女裝粉紅外套"
            all_dict[strWList[4]] = strWList[5]#把"gender"當成all_dict的key；對應的value為"w"
            all_dict[strWList[6]] = strWList[7]#把"price"當成all_dict的key；對應的value為""600"
            all_dict[strWList[8]] = strWList[9]#把"website_url"當成all_dict的key；對應的value為"https://www.uniqlo.com/tw/store/goods/436738"
        
        t = int( (len(strList) - 2)/7 )#有幾個item跑的次數
        for i in range(0,t):
            #把"Class","Probability","Bounding box"當s的key
            #等號後是value:"trousers",0.6078929901123047,[0.1509296, 0.7262472, 0.738734, 1.0000496]
            s[strList[3+7*i]] = strList[4+7*i]
            s[strList[5+7*i]] = strList[6+7*i]
            s[strList[7+7*i]] = strList[8+7*i]
            # 判斷此分類器 json是哪一個 item的
            for n in range(0,t):
                if strCList[n*7] == s["'Class'"].replace("'",'').lstrip() :
                    #把"Classification","Highest_probability","Color"當s的key
                    #等號後是value:"floral",0.5591356158256531,"red"
                    s[strCList[7*n+1]] = strCList[7*n+2]
                    s[strCList[7*n+3]] = strCList[7*n+4]
                    s[strCList[7*n+5]] = strCList[7*n+6]
                    
            #把"item1","item2"當all_dict的key
            #等號後是value:"Class","Probability","Bounding box","Classification","Highest_probability","Color"
            all_dict[strList[2+7*i]] = s
            s = {}
        
        str_all_dict = str(all_dict).replace("'",'')
        all_dict = eval(str_all_dict)
        filename = org_name + '.json'
    
        with open(os.path.join(destination_dir,filename), "w",encoding = 'utf-8') as f:
            json.dump(all_dict, f,ensure_ascii=False,indent = 4) #不使用的ascii編碼，才不會讓中文字變亂碼
            
        strCList = []
        strWList = []
        strList = []
        all_dict = {}


#如果出現out of list之類的錯誤的話就是json file裡面有3個item，把機率最低的那個刪掉就好

["'brand'", "'NET'", "'product_name'", "'小恐龍印花T恤'", "'gender'", "'W'", "'price'", "'299'", "'website_url'", "'https", "//www.net-fashion.net/product/346603'"]
["'brand'", "'NET'", "'product_name'", "'滿版印花圓領T恤'", "'gender'", "'W'", "'price'", "'259'", "'website_url'", "'https", "//www.net-fashion.net/product/348064'"]
["'brand'", "'NET'", "'product_name'", "'滿版印花圓領T恤'", "'gender'", "'W'", "'price'", "'259'", "'website_url'", "'https", "//www.net-fashion.net/product/348064'"]
["'brand'", "'NET'", "'product_name'", "'滿版印花圓領T恤'", "'gender'", "'W'", "'price'", "'259'", "'website_url'", "'https", "//www.net-fashion.net/product/348064'"]
["'brand'", "'NET'", "'product_name'", "'滿版印花圓領T恤'", "'gender'", "'W'", "'price'", "'259'", "'website_url'", "'https", "//www.net-fashion.net/product/348064'"]
["'brand'", "'NET'", "'product_name'", "'滿版印花圓領T恤'", "'gender'", "'W'", "'price'", "'259'", "'website_url'", "'https", "//www.net-fashion.net/product/348064'"]
["'brand'", "'NET'", "'product_name'", "'

IndexError: list index out of range